Użyj każdą z tych funkcji 
* `unix_timestamp()` 
* `date_format()`
* `to_unix_timestamp()`
* `from_unixtime()`
* `to_date()` 
* `to_timestamp()` 
* `from_utc_timestamp()` 
* `to_utc_timestamp()`

In [0]:
from pyspark.sql.functions import current_date, current_timestamp

kolumny = ["timestamp", "unix", "Date"]
dane = [("2015-03-22T14:13:34", 1646641525847, "May, 2021"),
        ("2015-03-22T15:03:18", 1646641557555, "Mar, 2021"),
        ("2015-03-22T14:38:39", 1646641578622, "Jan, 2021")]

dataFrame = spark.createDataFrame(dane, kolumny) \
    .withColumn("current_date", current_date()) \
    .withColumn("current_timestamp", current_timestamp())

display(dataFrame)

timestamp,unix,Date,current_date,current_timestamp
2015-03-22T14:13:34,1646641525847,"May, 2021",2025-03-17,2025-03-17T16:07:44.894+0000
2015-03-22T15:03:18,1646641557555,"Mar, 2021",2025-03-17,2025-03-17T16:07:44.894+0000
2015-03-22T14:38:39,1646641578622,"Jan, 2021",2025-03-17,2025-03-17T16:07:44.894+0000


In [0]:

dataFrame.printSchema()

root
 |-- timestamp: string (nullable = true)
 |-- unix: long (nullable = true)
 |-- Date: string (nullable = true)
 |-- current_date: date (nullable = false)
 |-- current_timestamp: timestamp (nullable = false)



## unix_timestamp(..) & cast(..)

Konwersja **string** to a **timestamp**.

Lokalizacja funkcji 
* `pyspark.sql.functions` in the case of Python
* `org.apache.spark.sql.functions` in the case of Scala & Java

## 1. Zmiana formatu wartości timestamp yyyy-MM-dd'T'HH:mm:ss 
`unix_timestamp(..)`

Dokumentacja API `unix_timestamp(..)`:
> Convert time string with given pattern (see <a href="http://docs.oracle.com/javase/tutorial/i18n/format/simpleDateFormat.html" target="_blank">SimpleDateFormat</a>) to Unix time stamp (in seconds), return null if fail.

`SimpleDataFormat` is part of the Java API and provides support for parsing and formatting date and time values.

2. Zmień format zgodnie z klasą `SimpleDateFormat`**yyyy-MM-dd HH:mm:ss**
  * a. Wyświetl schemat i dane żeby sprawdzicz czy wartości się zmieniły

In [0]:
from pyspark.sql.functions import col, to_timestamp, unix_timestamp

zmianaFormatu = dataFrame.withColumn("timestamp", to_timestamp(col("timestamp"), "yyyy-MM-dd'T'HH:mm:ss"))

zmianaFormatu.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- unix: long (nullable = true)
 |-- Date: string (nullable = true)
 |-- current_date: date (nullable = false)
 |-- current_timestamp: timestamp (nullable = false)



In [0]:
#unix_timestamp
tempE = zmianaFormatu.select(unix_timestamp(col("timestamp")).alias("unix_timestamp"))

display(tempE)

unix_timestamp
1427033614
1427036598
1427035119


## Stwórz nowe kolumny do DataFrame z wartościami year(..), month(..), dayofyear(..)

In [0]:
from pyspark.sql.functions import year, month, dayofyear, date_format

# date_format
yearDate = dataFrame \
    .withColumn("year", date_format(col("timestamp"), "yyyy")) \
    .withColumn("month", date_format(col("timestamp"), "MM")) \
    .withColumn("day_of_year", date_format(col("timestamp"), "D"))

display(yearDate)


timestamp,unix,Date,current_date,current_timestamp,year,month,day_of_year
2015-03-22T14:13:34,1646641525847,"May, 2021",2025-03-17,2025-03-17T16:07:47.121+0000,2015,03,81
2015-03-22T15:03:18,1646641557555,"Mar, 2021",2025-03-17,2025-03-17T16:07:47.121+0000,2015,03,81
2015-03-22T14:38:39,1646641578622,"Jan, 2021",2025-03-17,2025-03-17T16:07:47.121+0000,2015,03,81


In [0]:
from pyspark.sql.functions import to_date, year, month, dayofyear, col

#to_date()
toDate = dataFrame  \
   .withColumn("date", to_date(col("timestamp"), "yyyy-MM-dd'T'HH:mm:ss")) \
    .withColumn("year", year(col("date"))) \
    .withColumn("month", month(col("date")))  \
  .withColumn("day_of_year", dayofyear(col("date")))
  
display(toDate)

timestamp,unix,date,current_date,current_timestamp,year,month,day_of_year
2015-03-22T14:13:34,1646641525847,2015-03-22,2025-03-17,2025-03-17T16:07:48.098+0000,2015,3,81
2015-03-22T15:03:18,1646641557555,2015-03-22,2025-03-17,2025-03-17T16:07:48.098+0000,2015,3,81
2015-03-22T14:38:39,1646641578622,2015-03-22,2025-03-17,2025-03-17T16:07:48.098+0000,2015,3,81


In [0]:
from pyspark.sql.functions import from_unixtime, year, month, dayofyear, col

#from_unixtime()
fromUnix = dataFrame \
    .withColumn("date_from_unix", from_unixtime(col("unix"), "yyyy-MM-dd HH:mm:ss")) \
    .withColumn("year", year(col("date_from_unix"))) \
    .withColumn("month", month(col("date_from_unix"))) \
    .withColumn("day_of_year", dayofyear(col("date_from_unix")))

display(fromUnix)


timestamp,unix,Date,current_date,current_timestamp,date_from_unix,year,month,day_of_year
2015-03-22T14:13:34,1646641525847,"May, 2021",2025-03-17,2025-03-17T16:07:49.287+0000,+54149-12-28 23:50:47,54149,12,362
2015-03-22T15:03:18,1646641557555,"Mar, 2021",2025-03-17,2025-03-17T16:07:49.287+0000,+54149-12-29 08:39:15,54149,12,363
2015-03-22T14:38:39,1646641578622,"Jan, 2021",2025-03-17,2025-03-17T16:07:49.287+0000,+54149-12-29 14:30:22,54149,12,363


In [0]:
from pyspark.sql.functions import col, to_timestamp

#to_timestamp()
toTimestamp = dataFrame \
    .withColumn("to_timestamp", to_timestamp(col("timestamp"), "yyyy-MM-dd'T'HH:mm:ss"))

display(toTimestamp)

timestamp,unix,Date,current_date,current_timestamp,to_timestamp
2015-03-22T14:13:34,1646641525847,"May, 2021",2025-03-17,2025-03-17T16:07:50.264+0000,2015-03-22T14:13:34.000+0000
2015-03-22T15:03:18,1646641557555,"Mar, 2021",2025-03-17,2025-03-17T16:07:50.264+0000,2015-03-22T15:03:18.000+0000
2015-03-22T14:38:39,1646641578622,"Jan, 2021",2025-03-17,2025-03-17T16:07:50.264+0000,2015-03-22T14:38:39.000+0000


In [0]:
from pyspark.sql.functions import col, to_utc_timestamp

#to_utc_timestamp()
toUtcTimestamp = dataFrame \
    .withColumn("to_utc_timestamp", to_utc_timestamp(col("timestamp"), "UTC"))

display(toUtcTimestamp)


timestamp,unix,Date,current_date,current_timestamp,to_utc_timestamp
2015-03-22T14:13:34,1646641525847,"May, 2021",2025-03-17,2025-03-17T16:07:51.035+0000,2015-03-22T14:13:34.000+0000
2015-03-22T15:03:18,1646641557555,"Mar, 2021",2025-03-17,2025-03-17T16:07:51.035+0000,2015-03-22T15:03:18.000+0000
2015-03-22T14:38:39,1646641578622,"Jan, 2021",2025-03-17,2025-03-17T16:07:51.035+0000,2015-03-22T14:38:39.000+0000


In [0]:
from pyspark.sql.functions import col, from_utc_timestamp

#from_utc_timestamp()
fromUtcTimestamp = dataFrame \
    .withColumn("from_utc_timestamp", from_utc_timestamp(col("timestamp"), "UTC"))

display(fromUtcTimestamp)

timestamp,unix,Date,current_date,current_timestamp,from_utc_timestamp
2015-03-22T14:13:34,1646641525847,"May, 2021",2025-03-17,2025-03-17T16:07:51.848+0000,2015-03-22T14:13:34.000+0000
2015-03-22T15:03:18,1646641557555,"Mar, 2021",2025-03-17,2025-03-17T16:07:51.848+0000,2015-03-22T15:03:18.000+0000
2015-03-22T14:38:39,1646641578622,"Jan, 2021",2025-03-17,2025-03-17T16:07:51.848+0000,2015-03-22T14:38:39.000+0000


In [0]:
%fs ls dbfs:/databricks-datasets/
